In [1]:
import pandas as pd
from os import path, environ, makedirs
from dotenv import load_dotenv
from unidecode import unidecode

In [2]:
load_dotenv()

True

# Carregando os dados extraídos no notebook anterior

Neste notebook, vamos utilizar os dados extraídos e salvos pelo notebook `03 habitação - extração.ipynb`.

In [3]:
input_dir = path.join('dados', 'urbanismo')

In [4]:
filename = path.join(input_dir, 'orcamento_habitacao_original.csv')
df_orcamento = pd.read_csv(filename,
            sep=';',
            decimal=',',
            encoding='utf8',
            dtype=str)
df_orcamento

,DataInicial,DataFinal,Cd_AnoExecucao,Cd_Exercicio,Cd_Dotacao_Id,Administracao,Cd_Orgao,Sigla_Orgao,Ds_Orgao,Cd_Unidade,...,Vl_Congelado,Vl_Descongelado,Vl_CongeladoLiquido,Disponivel,Vl_ReservadoLiquido,Vl_EmpenhadoLiquido,Vl_Liquidado,Vl_Pago,Saldo_Dotacao,DataExtracao
0,01/01/2024,31/12/2024,2024,2024,169113,Direta,07,FMD,Fundo Municipal de Desenvolvimento Social,10,...,0.0,0.0,0.0,1000,0.0,0.0,0.0,0.0,1000,2025-01-18 00:00:00
1,01/01/2024,31/12/2024,2024,2024,167099,Direta,07,FMD,Fundo Municipal de Desenvolvimento Social,10,...,0.0,0.0,0.0,1000,0.0,0.0,0.0,0.0,1000,2025-01-18 00:00:00
2,01/01/2024,31/12/2024,2024,2024,167101,Direta,07,FMD,Fundo Municipal de Desenvolvimento Social,10,...,0.0,0.0,0.0,1000,0.0,0.0,0.0,0.0,1000,2025-01-18 00:00:00
3,01/01/2024,31/12/2024,2024,2024,171051,Direta,07,FMD,Fundo Municipal de Desenvolvimento Social,10,...,0.0,0.0,0.0,14985177,14963773.14,14963773.14,14842626.58,14828557.72,"21403,86",2025-01-18 00:00:00
4,01/01/2024,31/12/2024,2024,2024,173960,Direta,07,FMD,Fundo Municipal de Desenvolvimento Social,10,...,0.0,0.0,0.0,10000000,9968965.45,9968965.45,6827361.22,6827361.22,"31034,55",2025-01-18 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
391,01/01/2024,31/12/2024,2024,2024,182123,Indireta,91,FMH,Fundo Municipal de Habitação,10,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,2025-01-18 00:00:00
392,01/01/2024,31/12/2024,2024,2024,182125,Indireta,91,FMH,Fundo Municipal de Habitação,10,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,2025-01-18 00:00:00
393,01/01/2024,31/12/2024,2024,2024,164701,Indireta,91,FMH,Fundo Municipal de Habitação,10,...,2745125.68,0.0,2745125.68,"10311885,81",6264151.26,6264151.26,5565823.14,5492338.9,"4047734,55",2025-01-18 00:00:00
394,01/01/2024,31/12/2024,2024,2024,182481,Indireta,91,FMH,Fundo Municipal de Habitação,10,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,2025-01-18 00:00:00


In [5]:
for col in [col for col in df_orcamento.columns if 'Vl' in col]:
    df_orcamento[col] = df_orcamento[col].astype(float)
df_orcamento['DataExtracao'] = pd.to_datetime(df_orcamento['DataExtracao'])
df_orcamento

,DataInicial,DataFinal,Cd_AnoExecucao,Cd_Exercicio,Cd_Dotacao_Id,Administracao,Cd_Orgao,Sigla_Orgao,Ds_Orgao,Cd_Unidade,...,Vl_Congelado,Vl_Descongelado,Vl_CongeladoLiquido,Disponivel,Vl_ReservadoLiquido,Vl_EmpenhadoLiquido,Vl_Liquidado,Vl_Pago,Saldo_Dotacao,DataExtracao
0,01/01/2024,31/12/2024,2024,2024,169113,Direta,07,FMD,Fundo Municipal de Desenvolvimento Social,10,...,0.00,0.0,0.00,1000,0.00,0.00,0.00,0.00,1000,2025-01-18
1,01/01/2024,31/12/2024,2024,2024,167099,Direta,07,FMD,Fundo Municipal de Desenvolvimento Social,10,...,0.00,0.0,0.00,1000,0.00,0.00,0.00,0.00,1000,2025-01-18
2,01/01/2024,31/12/2024,2024,2024,167101,Direta,07,FMD,Fundo Municipal de Desenvolvimento Social,10,...,0.00,0.0,0.00,1000,0.00,0.00,0.00,0.00,1000,2025-01-18
3,01/01/2024,31/12/2024,2024,2024,171051,Direta,07,FMD,Fundo Municipal de Desenvolvimento Social,10,...,0.00,0.0,0.00,14985177,14963773.14,14963773.14,14842626.58,14828557.72,"21403,86",2025-01-18
4,01/01/2024,31/12/2024,2024,2024,173960,Direta,07,FMD,Fundo Municipal de Desenvolvimento Social,10,...,0.00,0.0,0.00,10000000,9968965.45,9968965.45,6827361.22,6827361.22,"31034,55",2025-01-18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
391,01/01/2024,31/12/2024,2024,2024,182123,Indireta,91,FMH,Fundo Municipal de Habitação,10,...,0.00,0.0,0.00,0,0.00,0.00,0.00,0.00,0,2025-01-18
392,01/01/2024,31/12/2024,2024,2024,182125,Indireta,91,FMH,Fundo Municipal de Habitação,10,...,0.00,0.0,0.00,0,0.00,0.00,0.00,0.00,0,2025-01-18
393,01/01/2024,31/12/2024,2024,2024,164701,Indireta,91,FMH,Fundo Municipal de Habitação,10,...,2745125.68,0.0,2745125.68,"10311885,81",6264151.26,6264151.26,5565823.14,5492338.90,"4047734,55",2025-01-18
394,01/01/2024,31/12/2024,2024,2024,182481,Indireta,91,FMH,Fundo Municipal de Habitação,10,...,0.00,0.0,0.00,0,0.00,0.00,0.00,0.00,0,2025-01-18


In [6]:
filename = path.join(input_dir, 'orcamento_regionalizado_habitacao_original.csv')
df_orcamento_r = pd.read_csv(filename,
            sep=';',
            decimal=',',
            encoding='utf8',
            dtype=str)
df_orcamento_r

,COD_EMPRESA_PMSP,COD_EMPENHO,ANO_EMPENHO,CÓDIGO_NLP,ANO_LIQUIDAÇÃO,DATA_LIQUIDAÇÃO,CÓDIGO_ÓRGÃO,SIGLA_ÓRGÃO,DESCRIÇÃO_ÓRGÃO,CÓDIGO_UNIDADE,...,CÓDIGO_EXERCÍCIO_FONTE,CÓDIGO_DESTINAÇÃO_RECURSO,CÓDIGO_VÍNCULO_PMSP,CÓDIGO_TIPO_CRÉDITO_ORÇAMENTÁRIO,REGIÃO,SUBPREFEITURA,DISTRITO,TIPO_REGIONALIZAÇÃO,VALOR_DETALHAMENTO_AÇÃO,DATA_EXTRAÇÃO
0,01,41402,2024,91875,2024,2024-04-04 00:00:00.0000000,07,FMD,Fundo Municipal de Desenvolvimento Social,10,...,1,759,1224,0,Oeste,Subprefeitura Butantã,Supra-Distrital,Despesa Regionalizável,291886.62,2025-08-26 00:00:00
1,01,41402,2024,91887,2024,2024-04-04 00:00:00.0000000,07,FMD,Fundo Municipal de Desenvolvimento Social,10,...,1,759,1224,0,Oeste,Subprefeitura Butantã,Supra-Distrital,Despesa Regionalizável,396969.19,2025-08-26 00:00:00
2,01,41402,2024,142475,2024,2024-05-24 00:00:00.0000000,07,FMD,Fundo Municipal de Desenvolvimento Social,10,...,1,759,1224,0,Oeste,Subprefeitura Butantã,Supra-Distrital,Despesa Regionalizável,50401.4,2025-08-26 00:00:00
3,01,41402,2024,217562,2024,2024-08-13 00:00:00.0000000,07,FMD,Fundo Municipal de Desenvolvimento Social,10,...,1,759,1224,0,Oeste,Subprefeitura Butantã,Supra-Distrital,Despesa Regionalizável,132575.22,2025-08-26 00:00:00
4,01,41402,2024,270072,2024,2024-09-27 00:00:00.0000000,07,FMD,Fundo Municipal de Desenvolvimento Social,10,...,1,759,1224,0,Oeste,Subprefeitura Butantã,Supra-Distrital,Despesa Regionalizável,8541.96,2025-08-26 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6614,01,80617,2024,210865,2024,2024-08-06 00:00:00.0000000,98,FUNDURB,Fundo de Desenvolvimento Urbano,14,...,2,759,0402,1,Sul,Subprefeitura Capela do Socorro,Supra-Distrital,Despesa Regionalizável,2578610.44,2025-08-26 00:00:00
6615,01,84580,2024,210866,2024,2024-08-06 00:00:00.0000000,98,FUNDURB,Fundo de Desenvolvimento Urbano,14,...,2,759,0402,1,Sul,Subprefeitura Capela do Socorro,Supra-Distrital,Despesa Regionalizável,499937.05,2025-08-26 00:00:00
6616,01,84580,2024,272023,2024,2024-10-01 00:00:00.0000000,98,FUNDURB,Fundo de Desenvolvimento Urbano,14,...,2,759,0402,1,Sul,Subprefeitura Capela do Socorro,Supra-Distrital,Despesa Regionalizável,78673.39,2025-08-26 00:00:00
6617,01,120758,2024,297165,2024,2024-10-26 00:00:00.0000000,98,FUNDURB,Fundo de Desenvolvimento Urbano,14,...,2,759,0402,1,Sul,Subprefeitura Capela do Socorro,Supra-Distrital,Despesa Regionalizável,6780.25,2025-08-26 00:00:00


In [7]:
df_orcamento_r['VALOR_DETALHAMENTO_AÇÃO'] = df_orcamento_r['VALOR_DETALHAMENTO_AÇÃO'].astype(float)
df_orcamento_r['DATA_EXTRAÇÃO'] = pd.to_datetime(df_orcamento_r['DATA_EXTRAÇÃO'])
df_orcamento_r

,COD_EMPRESA_PMSP,COD_EMPENHO,ANO_EMPENHO,CÓDIGO_NLP,ANO_LIQUIDAÇÃO,DATA_LIQUIDAÇÃO,CÓDIGO_ÓRGÃO,SIGLA_ÓRGÃO,DESCRIÇÃO_ÓRGÃO,CÓDIGO_UNIDADE,...,CÓDIGO_EXERCÍCIO_FONTE,CÓDIGO_DESTINAÇÃO_RECURSO,CÓDIGO_VÍNCULO_PMSP,CÓDIGO_TIPO_CRÉDITO_ORÇAMENTÁRIO,REGIÃO,SUBPREFEITURA,DISTRITO,TIPO_REGIONALIZAÇÃO,VALOR_DETALHAMENTO_AÇÃO,DATA_EXTRAÇÃO
0,01,41402,2024,91875,2024,2024-04-04 00:00:00.0000000,07,FMD,Fundo Municipal de Desenvolvimento Social,10,...,1,759,1224,0,Oeste,Subprefeitura Butantã,Supra-Distrital,Despesa Regionalizável,291886.62,2025-08-26
1,01,41402,2024,91887,2024,2024-04-04 00:00:00.0000000,07,FMD,Fundo Municipal de Desenvolvimento Social,10,...,1,759,1224,0,Oeste,Subprefeitura Butantã,Supra-Distrital,Despesa Regionalizável,396969.19,2025-08-26
2,01,41402,2024,142475,2024,2024-05-24 00:00:00.0000000,07,FMD,Fundo Municipal de Desenvolvimento Social,10,...,1,759,1224,0,Oeste,Subprefeitura Butantã,Supra-Distrital,Despesa Regionalizável,50401.40,2025-08-26
3,01,41402,2024,217562,2024,2024-08-13 00:00:00.0000000,07,FMD,Fundo Municipal de Desenvolvimento Social,10,...,1,759,1224,0,Oeste,Subprefeitura Butantã,Supra-Distrital,Despesa Regionalizável,132575.22,2025-08-26
4,01,41402,2024,270072,2024,2024-09-27 00:00:00.0000000,07,FMD,Fundo Municipal de Desenvolvimento Social,10,...,1,759,1224,0,Oeste,Subprefeitura Butantã,Supra-Distrital,Despesa Regionalizável,8541.96,2025-08-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6614,01,80617,2024,210865,2024,2024-08-06 00:00:00.0000000,98,FUNDURB,Fundo de Desenvolvimento Urbano,14,...,2,759,0402,1,Sul,Subprefeitura Capela do Socorro,Supra-Distrital,Despesa Regionalizável,2578610.44,2025-08-26
6615,01,84580,2024,210866,2024,2024-08-06 00:00:00.0000000,98,FUNDURB,Fundo de Desenvolvimento Urbano,14,...,2,759,0402,1,Sul,Subprefeitura Capela do Socorro,Supra-Distrital,Despesa Regionalizável,499937.05,2025-08-26
6616,01,84580,2024,272023,2024,2024-10-01 00:00:00.0000000,98,FUNDURB,Fundo de Desenvolvimento Urbano,14,...,2,759,0402,1,Sul,Subprefeitura Capela do Socorro,Supra-Distrital,Despesa Regionalizável,78673.39,2025-08-26
6617,01,120758,2024,297165,2024,2024-10-26 00:00:00.0000000,98,FUNDURB,Fundo de Desenvolvimento Urbano,14,...,2,759,0402,1,Sul,Subprefeitura Capela do Socorro,Supra-Distrital,Despesa Regionalizável,6780.25,2025-08-26


In [8]:
df_orcamento_r.dtypes

COD_EMPRESA_PMSP                            object
COD_EMPENHO                                 object
ANO_EMPENHO                                 object
CÓDIGO_NLP                                  object
ANO_LIQUIDAÇÃO                              object
DATA_LIQUIDAÇÃO                             object
CÓDIGO_ÓRGÃO                                object
SIGLA_ÓRGÃO                                 object
DESCRIÇÃO_ÓRGÃO                             object
CÓDIGO_UNIDADE                              object
DESCRIÇÃO_UNIDADE                           object
CÓDIGO_FUNÇÃO                               object
DESCRIÇÃO_FUNÇÃO                            object
CÓDIGO_SUBFUNÇÃO                            object
DESCRIÇÃO_SUBFUNÇÃO                         object
CÓDIGO_PROGRAMA                             object
DESCRIÇÃO_PROGRAMA                          object
CÓDIGO_PROJ_ATIV                            object
DESCRIÇÃO_PROJ_ATIV                         object
CÓDIGO_CONTA_DESPESA           

In [9]:
filename = path.join(input_dir, 'his_entregue_original.csv')
df_his = pd.read_csv(filename,
            sep=';',
            decimal=',',
            encoding='utf8')
df_his

,região,indicador,ano,qtd_unidades
0,Campo Limpo,11.01.03 Número de Unidades Habitacionais entr...,2021,0.0
1,Capela Do Socorro,11.01.03 Número de Unidades Habitacionais entr...,2021,562.0
2,Ermelino Matarazzo,11.01.03 Número de Unidades Habitacionais entr...,2021,0.0
3,Freguesia-Brasilândia,11.01.03 Número de Unidades Habitacionais entr...,2021,228.0
4,Guaianases,11.01.03 Número de Unidades Habitacionais entr...,2021,0.0
...,...,...,...,...
91,Butantã,11.01.03 Número de Unidades Habitacionais entr...,2023,122.0
92,Aricanduva-Formosa-Carrão,11.01.03 Número de Unidades Habitacionais entr...,2023,0.0
93,Casa Verde-Cachoeirinha,11.01.03 Número de Unidades Habitacionais entr...,2023,20.0
94,Cidade Ademar,11.01.03 Número de Unidades Habitacionais entr...,2023,70.0


In [10]:
filename = path.join(input_dir, 'tpu_emitido_original.csv')
df_tpu = pd.read_csv(filename,
            sep=';',
            decimal=',',
            encoding='utf8')
df_tpu

,indicador,ano,qtd_termos
0,05.0a.04 Número de termos de Permissão de Uso ...,2021,285.0
1,05.0a.04 Número de termos de Permissão de Uso ...,2022,293.0
2,05.0a.04 Número de termos de Permissão de Uso ...,2023,384.0


In [11]:
filename = path.join(input_dir, 'indicador_11_01_07_2024.csv')
df_110107 = pd.read_csv(filename, sep=";", decimal=",", encoding="utf-8")
df_110107

,Subprefeitura,Domicílios em favelas (2024),ano
0,Campo Limpo,14,2024
1,Capela Do Socorro,17,2024
2,Ermelino Matarazzo,1,2024
3,Freguesia-Brasilândia,0,2024
4,Guaianases,3,2024
5,Ipiranga,240,2024
6,Itaim Paulista,1,2024
7,Itaquera,5,2024
8,Jabaquara,0,2024
9,Jaçanã-Tremembé,2,2024


# Transformação e padronização

Nos indicadores de habitação, apenas o ano e as subprefeituras são presentes em vários arquivos. Mas, além disso, os arquivos precisarão de tratamentos específicos. Vamos começar com os mais simples, onde é necessário apenas padronizar as subprefeituras.

Primeiro, vamos carregar os dados de subprefeituras que serão utilizados no Qlik Sense.

## CSV de Subprefeituras do Qlik

In [12]:
url_subs = environ.get('CSV_SUBPREFEITURAS_QLIK')
df_subs = pd.read_csv(url_subs)
df_subs

,sub.CODIGO,sub.NOME,sub.POLIGONO,total_de_pessoas
0,1,PERUS,"[[[[-46.7082012617476,-23.4165818252815],[-46....",163076
1,2,PIRITUBA-JARAGUA,"[[[[-46.7753669019781,-23.4628032705496],[-46....",480225
2,3,FREGUESIA-BRASILANDIA,"[[[[-46.6910223492151,-23.5088410763715],[-46....",380513
3,4,CASA VERDE-CACHOEIRINHA,"[[[[-46.673577305455,-23.479858692024],[-46.67...",306275
4,5,SANTANA-TUCURUVI,"[[[[-46.6251942372966,-23.5193645522374],[-46....",318913
5,6,JACANA-TREMEMBE,"[[[[-46.6138088697146,-23.4620183373337],[-46....",283892
6,7,VILA MARIA-VILA GUILHERME,"[[[[-46.5985584306269,-23.4916579527685],[-46....",276069
7,8,LAPA,"[[[[-46.7475634318888,-23.5587659797062],[-46....",338347
8,9,SE,"[[[[-46.6634017492882,-23.5366395157441],[-46....",423536
9,10,BUTANTA,"[[[[-46.7579053376809,-23.5510466758162],[-46....",468522


In [13]:
df_subs = df_subs[['sub.CODIGO', 'sub.NOME']]
df_subs

,sub.CODIGO,sub.NOME
0,1,PERUS
1,2,PIRITUBA-JARAGUA
2,3,FREGUESIA-BRASILANDIA
3,4,CASA VERDE-CACHOEIRINHA
4,5,SANTANA-TUCURUVI
5,6,JACANA-TREMEMBE
6,7,VILA MARIA-VILA GUILHERME
7,8,LAPA
8,9,SE
9,10,BUTANTA


## Chave composta subprefeitura-ano

Como 3 tabelas possuem valores para mais de um ano, também vale a pena a criação de uma chave composta entre subprefeitura e ano. A tabela que possui mais períodos é a tabela da meta 12 do Programa de Metas, com os anos de 2021, 2022, 2023 e 2024. Vamos criar uma tabela com o produto cartesiano entre subprefeituras e anos.

In [14]:
df_subs_ano = (
    df_subs[['sub.NOME']]
    .merge(pd.Series(data=[2021, 2022, 2023, 2024], name='ano'),
           how='cross')
)

df_subs_ano.loc[:, 'subprefeitura-ano'] = (
    df_subs_ano.loc[:, 'sub.NOME'] + ' | ' + df_subs_ano.loc[:, 'ano'].astype(str)
)

df_subs_ano

,sub.NOME,ano,subprefeitura-ano
0,PERUS,2021,PERUS | 2021
1,PERUS,2022,PERUS | 2022
2,PERUS,2023,PERUS | 2023
3,PERUS,2024,PERUS | 2024
4,PIRITUBA-JARAGUA,2021,PIRITUBA-JARAGUA | 2021
...,...,...,...
123,CIDADE TIRADENTES,2024,CIDADE TIRADENTES | 2024
124,SAPOPEMBA,2021,SAPOPEMBA | 2021
125,SAPOPEMBA,2022,SAPOPEMBA | 2022
126,SAPOPEMBA,2023,SAPOPEMBA | 2023


## Subprefeituras Qlik

In [15]:
subs_qlik = df_subs['sub.NOME'].unique().tolist()
subs_qlik.sort()
subs_qlik

['ARICANDUVA-FORMOSA-CARRAO',
 'BUTANTA',
 'CAMPO LIMPO',
 'CAPELA DO SOCORRO',
 'CASA VERDE-CACHOEIRINHA',
 'CIDADE ADEMAR',
 'CIDADE TIRADENTES',
 'ERMELINO MATARAZZO',
 'FREGUESIA-BRASILANDIA',
 'GUAIANASES',
 'IPIRANGA',
 'ITAIM PAULISTA',
 'ITAQUERA',
 'JABAQUARA',
 'JACANA-TREMEMBE',
 'LAPA',
 'M BOI MIRIM',
 'MOOCA',
 'PARELHEIROS',
 'PENHA',
 'PERUS',
 'PINHEIROS',
 'PIRITUBA-JARAGUA',
 'SANTANA-TUCURUVI',
 'SANTO AMARO',
 'SAO MATEUS',
 'SAO MIGUEL',
 'SAPOPEMBA',
 'SE',
 'VILA MARIA-VILA GUILHERME',
 'VILA MARIANA',
 'VILA PRUDENTE']

## Produção de habitação de interesse social

In [16]:
df_his

,região,indicador,ano,qtd_unidades
0,Campo Limpo,11.01.03 Número de Unidades Habitacionais entr...,2021,0.0
1,Capela Do Socorro,11.01.03 Número de Unidades Habitacionais entr...,2021,562.0
2,Ermelino Matarazzo,11.01.03 Número de Unidades Habitacionais entr...,2021,0.0
3,Freguesia-Brasilândia,11.01.03 Número de Unidades Habitacionais entr...,2021,228.0
4,Guaianases,11.01.03 Número de Unidades Habitacionais entr...,2021,0.0
...,...,...,...,...
91,Butantã,11.01.03 Número de Unidades Habitacionais entr...,2023,122.0
92,Aricanduva-Formosa-Carrão,11.01.03 Número de Unidades Habitacionais entr...,2023,0.0
93,Casa Verde-Cachoeirinha,11.01.03 Número de Unidades Habitacionais entr...,2023,20.0
94,Cidade Ademar,11.01.03 Número de Unidades Habitacionais entr...,2023,70.0


In [17]:
subs_his = df_his['região'].apply(unidecode).unique().tolist()
subs_his.sort()
subs_his

['Aricanduva-Formosa-Carrao',
 'Butanta',
 'Campo Limpo',
 'Capela Do Socorro',
 'Casa Verde-Cachoeirinha',
 'Cidade Ademar',
 'Cidade Tiradentes',
 'Ermelino Matarazzo',
 'Freguesia-Brasilandia',
 'Guaianases',
 'Ipiranga',
 'Itaim Paulista',
 'Itaquera',
 'Jabaquara',
 'Jacana-Tremembe',
 'Lapa',
 "M'Boi Mirim",
 'Mooca',
 'Parelheiros',
 'Penha',
 'Perus',
 'Pinheiros',
 'Pirituba-Jaragua',
 'Santana-Tucuruvi',
 'Santo Amaro',
 'Sao Mateus',
 'Sao Miguel',
 'Sapopemba',
 'Se',
 'Vila Maria-Vila Guilherme',
 'Vila Mariana',
 'Vila Prudente']

In [18]:
mapper_his = {
    s: q for s, q in zip(subs_his, subs_qlik)
}

mapper_his

{'Aricanduva-Formosa-Carrao': 'ARICANDUVA-FORMOSA-CARRAO',
 'Butanta': 'BUTANTA',
 'Campo Limpo': 'CAMPO LIMPO',
 'Capela Do Socorro': 'CAPELA DO SOCORRO',
 'Casa Verde-Cachoeirinha': 'CASA VERDE-CACHOEIRINHA',
 'Cidade Ademar': 'CIDADE ADEMAR',
 'Cidade Tiradentes': 'CIDADE TIRADENTES',
 'Ermelino Matarazzo': 'ERMELINO MATARAZZO',
 'Freguesia-Brasilandia': 'FREGUESIA-BRASILANDIA',
 'Guaianases': 'GUAIANASES',
 'Ipiranga': 'IPIRANGA',
 'Itaim Paulista': 'ITAIM PAULISTA',
 'Itaquera': 'ITAQUERA',
 'Jabaquara': 'JABAQUARA',
 'Jacana-Tremembe': 'JACANA-TREMEMBE',
 'Lapa': 'LAPA',
 "M'Boi Mirim": 'M BOI MIRIM',
 'Mooca': 'MOOCA',
 'Parelheiros': 'PARELHEIROS',
 'Penha': 'PENHA',
 'Perus': 'PERUS',
 'Pinheiros': 'PINHEIROS',
 'Pirituba-Jaragua': 'PIRITUBA-JARAGUA',
 'Santana-Tucuruvi': 'SANTANA-TUCURUVI',
 'Santo Amaro': 'SANTO AMARO',
 'Sao Mateus': 'SAO MATEUS',
 'Sao Miguel': 'SAO MIGUEL',
 'Sapopemba': 'SAPOPEMBA',
 'Se': 'SE',
 'Vila Maria-Vila Guilherme': 'VILA MARIA-VILA GUILHERME',


In [19]:
df_his.insert(1,
                  'sub.NOME',
                  df_his['região'].apply(unidecode).map(mapper_his))
df_his

,região,sub.NOME,indicador,ano,qtd_unidades
0,Campo Limpo,CAMPO LIMPO,11.01.03 Número de Unidades Habitacionais entr...,2021,0.0
1,Capela Do Socorro,CAPELA DO SOCORRO,11.01.03 Número de Unidades Habitacionais entr...,2021,562.0
2,Ermelino Matarazzo,ERMELINO MATARAZZO,11.01.03 Número de Unidades Habitacionais entr...,2021,0.0
3,Freguesia-Brasilândia,FREGUESIA-BRASILANDIA,11.01.03 Número de Unidades Habitacionais entr...,2021,228.0
4,Guaianases,GUAIANASES,11.01.03 Número de Unidades Habitacionais entr...,2021,0.0
...,...,...,...,...,...
91,Butantã,BUTANTA,11.01.03 Número de Unidades Habitacionais entr...,2023,122.0
92,Aricanduva-Formosa-Carrão,ARICANDUVA-FORMOSA-CARRAO,11.01.03 Número de Unidades Habitacionais entr...,2023,0.0
93,Casa Verde-Cachoeirinha,CASA VERDE-CACHOEIRINHA,11.01.03 Número de Unidades Habitacionais entr...,2023,20.0
94,Cidade Ademar,CIDADE ADEMAR,11.01.03 Número de Unidades Habitacionais entr...,2023,70.0


In [20]:
df_his['qtd_unidades'] = df_his['qtd_unidades'].astype(int)
df_his['ano'] = df_his['ano'].astype(int)
df_his

,região,sub.NOME,indicador,ano,qtd_unidades
0,Campo Limpo,CAMPO LIMPO,11.01.03 Número de Unidades Habitacionais entr...,2021,0
1,Capela Do Socorro,CAPELA DO SOCORRO,11.01.03 Número de Unidades Habitacionais entr...,2021,562
2,Ermelino Matarazzo,ERMELINO MATARAZZO,11.01.03 Número de Unidades Habitacionais entr...,2021,0
3,Freguesia-Brasilândia,FREGUESIA-BRASILANDIA,11.01.03 Número de Unidades Habitacionais entr...,2021,228
4,Guaianases,GUAIANASES,11.01.03 Número de Unidades Habitacionais entr...,2021,0
...,...,...,...,...,...
91,Butantã,BUTANTA,11.01.03 Número de Unidades Habitacionais entr...,2023,122
92,Aricanduva-Formosa-Carrão,ARICANDUVA-FORMOSA-CARRAO,11.01.03 Número de Unidades Habitacionais entr...,2023,0
93,Casa Verde-Cachoeirinha,CASA VERDE-CACHOEIRINHA,11.01.03 Número de Unidades Habitacionais entr...,2023,20
94,Cidade Ademar,CIDADE ADEMAR,11.01.03 Número de Unidades Habitacionais entr...,2023,70


In [21]:
df_his = df_his.merge(df_subs_ano,
                              how='left',
                              on=['sub.NOME', 'ano'])

df_his

,região,sub.NOME,indicador,ano,qtd_unidades,subprefeitura-ano
0,Campo Limpo,CAMPO LIMPO,11.01.03 Número de Unidades Habitacionais entr...,2021,0,CAMPO LIMPO | 2021
1,Capela Do Socorro,CAPELA DO SOCORRO,11.01.03 Número de Unidades Habitacionais entr...,2021,562,CAPELA DO SOCORRO | 2021
2,Ermelino Matarazzo,ERMELINO MATARAZZO,11.01.03 Número de Unidades Habitacionais entr...,2021,0,ERMELINO MATARAZZO | 2021
3,Freguesia-Brasilândia,FREGUESIA-BRASILANDIA,11.01.03 Número de Unidades Habitacionais entr...,2021,228,FREGUESIA-BRASILANDIA | 2021
4,Guaianases,GUAIANASES,11.01.03 Número de Unidades Habitacionais entr...,2021,0,GUAIANASES | 2021
...,...,...,...,...,...,...
91,Butantã,BUTANTA,11.01.03 Número de Unidades Habitacionais entr...,2023,122,BUTANTA | 2023
92,Aricanduva-Formosa-Carrão,ARICANDUVA-FORMOSA-CARRAO,11.01.03 Número de Unidades Habitacionais entr...,2023,0,ARICANDUVA-FORMOSA-CARRAO | 2023
93,Casa Verde-Cachoeirinha,CASA VERDE-CACHOEIRINHA,11.01.03 Número de Unidades Habitacionais entr...,2023,20,CASA VERDE-CACHOEIRINHA | 2023
94,Cidade Ademar,CIDADE ADEMAR,11.01.03 Número de Unidades Habitacionais entr...,2023,70,CIDADE ADEMAR | 2023


## Número de termos de Permissão de Uso (TPU) emitidos em nome da mulher da familia

In [22]:
df_tpu['qtd_termos'] = df_tpu['qtd_termos'].astype(int)
df_tpu['ano'] = df_tpu['ano'].astype(int)
df_tpu

,indicador,ano,qtd_termos
0,05.0a.04 Número de termos de Permissão de Uso ...,2021,285
1,05.0a.04 Número de termos de Permissão de Uso ...,2022,293
2,05.0a.04 Número de termos de Permissão de Uso ...,2023,384


## Indicador 11.01.07

In [23]:
df_110107

,Subprefeitura,Domicílios em favelas (2024),ano
0,Campo Limpo,14,2024
1,Capela Do Socorro,17,2024
2,Ermelino Matarazzo,1,2024
3,Freguesia-Brasilândia,0,2024
4,Guaianases,3,2024
5,Ipiranga,240,2024
6,Itaim Paulista,1,2024
7,Itaquera,5,2024
8,Jabaquara,0,2024
9,Jaçanã-Tremembé,2,2024


In [24]:
subs_110107 = df_110107['Subprefeitura'].apply(unidecode).unique().tolist()
subs_110107.sort()
subs_110107

['Aricanduva-Formosa-Carrao',
 'Butanta',
 'Campo Limpo',
 'Capela Do Socorro',
 'Casa Verde-Cachoeirinha',
 'Cidade Ademar',
 'Cidade Tiradentes',
 'Ermelino Matarazzo',
 'Freguesia-Brasilandia',
 'Guaianases',
 'Ipiranga',
 'Itaim Paulista',
 'Itaquera',
 'Jabaquara',
 'Jacana-Tremembe',
 'Lapa',
 "M'Boi Mirim",
 'Mooca',
 'Parelheiros',
 'Penha',
 'Perus',
 'Pinheiros',
 'Pirituba-Jaragua',
 'Santana-Tucuruvi',
 'Santo Amaro',
 'Sao Mateus',
 'Sao Miguel',
 'Sapopemba',
 'Se',
 'Vila Maria-Vila Guilherme',
 'Vila Mariana',
 'Vila Prudente']

In [25]:
mapper_110107 = {
    s: q for s, q in zip(subs_110107, subs_qlik)
}

mapper_110107

{'Aricanduva-Formosa-Carrao': 'ARICANDUVA-FORMOSA-CARRAO',
 'Butanta': 'BUTANTA',
 'Campo Limpo': 'CAMPO LIMPO',
 'Capela Do Socorro': 'CAPELA DO SOCORRO',
 'Casa Verde-Cachoeirinha': 'CASA VERDE-CACHOEIRINHA',
 'Cidade Ademar': 'CIDADE ADEMAR',
 'Cidade Tiradentes': 'CIDADE TIRADENTES',
 'Ermelino Matarazzo': 'ERMELINO MATARAZZO',
 'Freguesia-Brasilandia': 'FREGUESIA-BRASILANDIA',
 'Guaianases': 'GUAIANASES',
 'Ipiranga': 'IPIRANGA',
 'Itaim Paulista': 'ITAIM PAULISTA',
 'Itaquera': 'ITAQUERA',
 'Jabaquara': 'JABAQUARA',
 'Jacana-Tremembe': 'JACANA-TREMEMBE',
 'Lapa': 'LAPA',
 "M'Boi Mirim": 'M BOI MIRIM',
 'Mooca': 'MOOCA',
 'Parelheiros': 'PARELHEIROS',
 'Penha': 'PENHA',
 'Perus': 'PERUS',
 'Pinheiros': 'PINHEIROS',
 'Pirituba-Jaragua': 'PIRITUBA-JARAGUA',
 'Santana-Tucuruvi': 'SANTANA-TUCURUVI',
 'Santo Amaro': 'SANTO AMARO',
 'Sao Mateus': 'SAO MATEUS',
 'Sao Miguel': 'SAO MIGUEL',
 'Sapopemba': 'SAPOPEMBA',
 'Se': 'SE',
 'Vila Maria-Vila Guilherme': 'VILA MARIA-VILA GUILHERME',


In [26]:
df_110107.insert(1,
                  'sub.NOME',
                  df_110107['Subprefeitura'].apply(unidecode).map(mapper_110107))
df_110107

,Subprefeitura,sub.NOME,Domicílios em favelas (2024),ano
0,Campo Limpo,CAMPO LIMPO,14,2024
1,Capela Do Socorro,CAPELA DO SOCORRO,17,2024
2,Ermelino Matarazzo,ERMELINO MATARAZZO,1,2024
3,Freguesia-Brasilândia,FREGUESIA-BRASILANDIA,0,2024
4,Guaianases,GUAIANASES,3,2024
5,Ipiranga,IPIRANGA,240,2024
6,Itaim Paulista,ITAIM PAULISTA,1,2024
7,Itaquera,ITAQUERA,5,2024
8,Jabaquara,JABAQUARA,0,2024
9,Jaçanã-Tremembé,JACANA-TREMEMBE,2,2024


In [27]:
df_110107['qtd_domicilios'] = df_110107['Domicílios em favelas (2024)'].astype(int)
df_110107['ano'] = df_110107['ano'].astype(int)
df_110107 = df_110107.drop(columns=['Domicílios em favelas (2024)'])
df_110107

,Subprefeitura,sub.NOME,ano,qtd_domicilios
0,Campo Limpo,CAMPO LIMPO,2024,14
1,Capela Do Socorro,CAPELA DO SOCORRO,2024,17
2,Ermelino Matarazzo,ERMELINO MATARAZZO,2024,1
3,Freguesia-Brasilândia,FREGUESIA-BRASILANDIA,2024,0
4,Guaianases,GUAIANASES,2024,3
5,Ipiranga,IPIRANGA,2024,240
6,Itaim Paulista,ITAIM PAULISTA,2024,1
7,Itaquera,ITAQUERA,2024,5
8,Jabaquara,JABAQUARA,2024,0
9,Jaçanã-Tremembé,JACANA-TREMEMBE,2024,2


In [28]:
df_110107 = df_110107.merge(df_subs_ano,
                              how='left',
                              on=['sub.NOME', 'ano'])

df_110107

,Subprefeitura,sub.NOME,ano,qtd_domicilios,subprefeitura-ano
0,Campo Limpo,CAMPO LIMPO,2024,14,CAMPO LIMPO | 2024
1,Capela Do Socorro,CAPELA DO SOCORRO,2024,17,CAPELA DO SOCORRO | 2024
2,Ermelino Matarazzo,ERMELINO MATARAZZO,2024,1,ERMELINO MATARAZZO | 2024
3,Freguesia-Brasilândia,FREGUESIA-BRASILANDIA,2024,0,FREGUESIA-BRASILANDIA | 2024
4,Guaianases,GUAIANASES,2024,3,GUAIANASES | 2024
5,Ipiranga,IPIRANGA,2024,240,IPIRANGA | 2024
6,Itaim Paulista,ITAIM PAULISTA,2024,1,ITAIM PAULISTA | 2024
7,Itaquera,ITAQUERA,2024,5,ITAQUERA | 2024
8,Jabaquara,JABAQUARA,2024,0,JABAQUARA | 2024
9,Jaçanã-Tremembé,JACANA-TREMEMBE,2024,2,JACANA-TREMEMBE | 2024


## Indicador 11.01.02

In [29]:
filename = path.join(input_dir, 'indicador_110102.csv')
df_110102 = pd.read_csv(filename, sep=';', decimal=',', encoding='utf8')
df_110102

,Unnamed: 0,região,nivel_regional,indicador,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,0,São Paulo,Município,Estimativa de domicílios em favela (%),10.7775,10.6451,10.6850,10.4328,10.4970,10.0282,9.9171,9.8971,9.7701,9.6454,9.5384,9.4386,8.0131
1,0,Penha,Subprefeitura,Estimativa de domicílios em favela (%),6.1051,6.0611,6.0936,6.0666,6.3761,6.3265,6.2849,6.6113,6.5684,6.5262,6.4844,6.4504,6.4167
2,1,Pinheiros,Subprefeitura,Estimativa de domicílios em favela (%),0.2528,0.2505,0.2482,0.2459,0.2436,0.2728,0.2713,0.2697,0.2682,0.2667,0.2653,0.2646,0.2640
3,2,Pirituba-Jaraguá,Subprefeitura,Estimativa de domicílios em favela (%),11.2209,11.0466,11.0093,11.0464,11.6688,12.2999,13.9439,11.4060,11.2385,11.0737,10.9111,10.7789,10.6484
4,3,Freguesia-Brasilândia,Subprefeitura,Estimativa de domicílios em favela (%),16.3806,16.5933,17.6115,17.0176,19.3485,19.0544,15.7627,18.6290,18.4202,18.2145,18.0115,17.8173,17.6342
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,91,Campo Grande,Distrito,Estimativa de domicílios em favela (%),4.1286,4.0503,3.8754,3.8161,3.7579,3.9487,3.8242,3.9065,3.8590,3.8123,3.7663,3.7299,3.6941
125,92,Cidade Líder,Distrito,Estimativa de domicílios em favela (%),8.9934,8.8348,7.5494,7.4272,7.2979,7.1711,13.8453,6.9712,6.8668,6.7643,6.6638,6.5799,7.4672
126,93,Cidade Tiradentes,Distrito,Estimativa de domicílios em favela (%),5.5894,5.4513,5.4500,4.8577,4.7386,4.5572,3.6508,4.4702,4.3689,4.2700,4.1774,4.0946,4.0135
127,94,Consolação,Distrito,Estimativa de domicílios em favela (%),0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000


In [30]:
# Padronizar os nomes das subprefeituras
subs_110102 = df_110102['região'].apply(unidecode).unique().tolist()
subs_110102.sort()

mapper_110102 = {
    s: q for s, q in zip(subs_110102, subs_qlik)
}

df_110102.insert(1,
                 'sub.NOME',
                 df_110102['região'].apply(unidecode).map(mapper_110102))
df_110102

,Unnamed: 0,sub.NOME,região,nivel_regional,indicador,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,0,NaN,São Paulo,Município,Estimativa de domicílios em favela (%),10.7775,10.6451,10.6850,10.4328,10.4970,10.0282,9.9171,9.8971,9.7701,9.6454,9.5384,9.4386,8.0131
1,0,NaN,Penha,Subprefeitura,Estimativa de domicílios em favela (%),6.1051,6.0611,6.0936,6.0666,6.3761,6.3265,6.2849,6.6113,6.5684,6.5262,6.4844,6.4504,6.4167
2,1,NaN,Pinheiros,Subprefeitura,Estimativa de domicílios em favela (%),0.2528,0.2505,0.2482,0.2459,0.2436,0.2728,0.2713,0.2697,0.2682,0.2667,0.2653,0.2646,0.2640
3,2,NaN,Pirituba-Jaraguá,Subprefeitura,Estimativa de domicílios em favela (%),11.2209,11.0466,11.0093,11.0464,11.6688,12.2999,13.9439,11.4060,11.2385,11.0737,10.9111,10.7789,10.6484
4,3,NaN,Freguesia-Brasilândia,Subprefeitura,Estimativa de domicílios em favela (%),16.3806,16.5933,17.6115,17.0176,19.3485,19.0544,15.7627,18.6290,18.4202,18.2145,18.0115,17.8173,17.6342
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,91,M BOI MIRIM,Campo Grande,Distrito,Estimativa de domicílios em favela (%),4.1286,4.0503,3.8754,3.8161,3.7579,3.9487,3.8242,3.9065,3.8590,3.8123,3.7663,3.7299,3.6941
125,92,SAO MIGUEL,Cidade Líder,Distrito,Estimativa de domicílios em favela (%),8.9934,8.8348,7.5494,7.4272,7.2979,7.1711,13.8453,6.9712,6.8668,6.7643,6.6638,6.5799,7.4672
126,93,SAPOPEMBA,Cidade Tiradentes,Distrito,Estimativa de domicílios em favela (%),5.5894,5.4513,5.4500,4.8577,4.7386,4.5572,3.6508,4.4702,4.3689,4.2700,4.1774,4.0946,4.0135
127,94,SE,Consolação,Distrito,Estimativa de domicílios em favela (%),0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000


In [31]:

# Transformar os anos em colunas para linhas
df_110102 = df_110102.melt(
    ['sub.NOME', 'região', 'indicador'],
    ['2022'],
    var_name='ano',
    value_name='qtd_familias'
)

# Ajustar os tipos de dados
df_110102['qtd_familias'] = df_110102['qtd_familias'].astype(int)
df_110102['ano'] = df_110102['ano'].astype(int)

# Adicionar a chave composta subprefeitura-ano
df_110102 = df_110102.merge(df_subs_ano,
                            how='left',
                            on=['sub.NOME', 'ano'])

df_110102

,sub.NOME,região,indicador,ano,qtd_familias,subprefeitura-ano
0,NaN,São Paulo,Estimativa de domicílios em favela (%),2022,8,NaN
1,NaN,Penha,Estimativa de domicílios em favela (%),2022,6,NaN
2,NaN,Pinheiros,Estimativa de domicílios em favela (%),2022,0,NaN
3,NaN,Pirituba-Jaraguá,Estimativa de domicílios em favela (%),2022,10,NaN
4,NaN,Freguesia-Brasilândia,Estimativa de domicílios em favela (%),2022,17,NaN
...,...,...,...,...,...,...
124,M BOI MIRIM,Campo Grande,Estimativa de domicílios em favela (%),2022,3,M BOI MIRIM | 2022
125,SAO MIGUEL,Cidade Líder,Estimativa de domicílios em favela (%),2022,7,SAO MIGUEL | 2022
126,SAPOPEMBA,Cidade Tiradentes,Estimativa de domicílios em favela (%),2022,4,SAPOPEMBA | 2022
127,SE,Consolação,Estimativa de domicílios em favela (%),2022,0,SE | 2022


# Armazenamento

Finalmente, vamos exportar os dados em formato csv compatível com o Qlik e no padrão do excel para português do Brasil.

In [ ]:
base_path = path.join('data_output', 'urbanismo')

if not path.exists(base_path):
    makedirs(base_path)

for name, df in [('producao-his', df_his),
                 ('emissoes-tpu', df_tpu),
                 ('subprefeitura-ano', df_subs_ano),
                 ('indicador_110107', df_110107),
                 ('indicador_110102', df_110102),
                 ]:

    filepath = path.join(base_path, f'{name}.csv')

    df.to_csv(filepath,
              index=False,
              sep=';',
              decimal=',',
              encoding='latin1')